In [27]:
library(tidyverse)
library(TCGAbiolinks)
library(HDF5Array)
library(SummarizedExperiment)

# Custom package
library(rutils)

Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Attaching package: ‘Biobase’

The following objects are masked from ‘package:matrixStats’:

    anyMissing, rowMedians



# Constants

In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-OV", "TCGA-UCS", "TCGA-UCEC")
project_paths <- unlist(map(projects, function(prj) paste0(dirs$data_dir, "/", prj)))
biolinks_dir <- "tcga_biolinks_downloads"
RSE_objects_dir <- "saved_RSE_objects"
TCGA_dest_dir <- paste0(dirs$data_dir, "/", biolinks_dir)
RSE_objects_dest_dir <- paste0(dirs$data_dir, "/", RSE_objects_dir)

# Functions

In [32]:
rna_seq_query <- function(p) {
    return(GDCquery(
        project = p,
        data.category = "Transcriptome Profiling",
        data.type = "Gene Expression Quantification",
        workflow.type = "HTSeq - Counts"
    ))
}


load_RSE_objects <- function(dir, projects, prefixes) {
    data_ls <- list()
    for (i in seq_len(length(projects))) {
        data_ls[[projects[i]]] <- loadHDF5SummarizedExperiment(dir = dir, prefix = prefixes[i])
    }
    return(data_ls)
}


prep_and_save_count_data <- function(rses, label_field, dest_dir, dest_subdir) {
    id_symbol_map <- as_tibble(rowData(rses[[1]]))
    
    # Does the matrix data directory exist? If not, create it
    dir.create(paste0(dest_dir, "/", dest_subdir))
    
    for (n in names(rses)) {
        counts_df <- assays(rses[[n]])[["HTSeq - Counts"]] %>%
            as_tibble(rownames = "ensembl_gene_id") %>%
            inner_join(id_symbol_map, by = "ensembl_gene_id") %>%
            dplyr::select(ensembl_gene_id, external_gene_name, everything()) %>%
            dplyr::select(-original_ensembl_gene_id)
        
        condition_labels <- rses[[n]][[label_field]]
        coldata_df <- as_tibble(colnames(counts_df)[-c(1:2)]) %>%
            dplyr::rename(sample_name = value) %>%
            mutate(condition = condition_labels) %>%
            mutate(project = n)
        
        write_tsv(counts_df, path = paste0(dest_dir, "/", dest_subdir, "/", n, "_counts.tsv"))
        write_tsv(coldata_df, path = paste0(dest_dir, "/", dest_subdir, "/", n, "_coldata.tsv"))
    }
}

# Download and save data
No loops used here since API is really finicky and quits abruptly

In [10]:
proj_idx <- 1
q <- rna_seq_query(projects[proj_idx])
GDCdownload(q, method = "api", directory = TCGA_dest_dir, files.per.chunk = 10)
data <- GDCprepare(q, directory = TCGA_dest_dir)
data
saveHDF5SummarizedExperiment(data, dir = RSE_objects_dest_dir, prefix = paste0(projects[proj_idx], "_RNA_"))

--------------------------------------
o GDCquery: Searching in GDC database
--------------------------------------
Genome of reference: hg38
--------------------------------------------
oo Accessing GDC. This might take a while...
--------------------------------------------
ooo Project: TCGA-CESC
--------------------
oo Filtering results
--------------------
ooo By data.type
ooo By workflow.type
----------------
oo Checking data
----------------
ooo Check if there are duplicated cases
ooo Check if there results for the query
-------------------
o Preparing output
-------------------
GDCdownload will download 309 files. A total of 77.948874 MB


Downloading: 2.5 MB       

Downloading: 2.5 MB       

Downloading: 2.5 MB       

Downloading: 2.5 MB       

...

Downloading: 2.5 MB       

Downloading: 2.5 MB       

|====================================================|100%                      Completed after 28 s 


Starting to add information to samples
 => Add clinical information to samples
Add FFPE information. More information at: 
=> https://cancergenome.nih.gov/cancersselected/biospeccriteria 
=> http://gdac.broadinstitute.org/runs/sampleReports/latest/FPPP_FFPE_Cases.html
 => Adding subtype information to samples
cesc subtype information from:doi:10.1038/nature21386
Accessing www.ensembl.org to get gene information
Warning message:
“`select_()` is deprecated as of dplyr 0.7.0.
Please use `select()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Warning message:
“`filter_()` is deprecated as of dplyr 0.7.0.
Please use `filter()` instead.
See vignette('programming') for more help
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Cache found
From the 60483 genes we couldn't map 3990


class: RangedSummarizedExperiment 
dim: 56493 309 
metadata(1): data_release
assays(1): HTSeq - Counts
rownames(56493): ENSG00000000003 ENSG00000000005 ... ENSG00000281912
  ENSG00000281920
rowData names(3): ensembl_gene_id external_gene_name
  original_ensembl_gene_id
colnames(309): TCGA-MY-A5BF-01A-11R-A26T-07
  TCGA-EA-A3HU-01A-11R-A213-07 ... TCGA-EK-A2R8-01A-21R-A18M-07
  TCGA-JW-A5VK-01A-11R-A28H-07
colData names(131): sample patient ... subtype_GEXP.APOBEC3H.164668
  subtype_patient

In [11]:
proj_idx <- 2
q <- rna_seq_query(projects[proj_idx])
GDCdownload(q, method = "api", directory = TCGA_dest_dir, files.per.chunk = 10)
data <- GDCprepare(q, directory = TCGA_dest_dir)
data
saveHDF5SummarizedExperiment(data, dir = RSE_objects_dest_dir, prefix = paste0(projects[proj_idx], "_RNA_"))

--------------------------------------
o GDCquery: Searching in GDC database
--------------------------------------
Genome of reference: hg38
--------------------------------------------
oo Accessing GDC. This might take a while...
--------------------------------------------
ooo Project: TCGA-OV
--------------------
oo Filtering results
--------------------
ooo By data.type
ooo By workflow.type
----------------
oo Checking data
----------------
ooo Check if there are duplicated cases
ooo Check if there results for the query
-------------------
o Preparing output
-------------------
GDCdownload will download 379 files. A total of 97.709866 MB


Downloading: 2.6 MB     

Downloading: 2.6 MB       

Downloading: 2.5 MB     

Downloading: 2.5 MB       

...

Downloading: 2.6 MB     

Downloading: 2.6 MB     

|====================================================|100%                      Completed after 33 s 


Starting to add information to samples
 => Add clinical information to samples
Add FFPE information. More information at: 
=> https://cancergenome.nih.gov/cancersselected/biospeccriteria 
=> http://gdac.broadinstitute.org/runs/sampleReports/latest/FPPP_FFPE_Cases.html
 => Adding subtype information to samples
Accessing www.ensembl.org to get gene information
Cache found
From the 60483 genes we couldn't map 3990


class: RangedSummarizedExperiment 
dim: 56493 379 
metadata(1): data_release
assays(1): HTSeq - Counts
rownames(56493): ENSG00000000003 ENSG00000000005 ... ENSG00000281912
  ENSG00000281920
rowData names(3): ensembl_gene_id external_gene_name
  original_ensembl_gene_id
colnames(379): TCGA-04-1338-01A-01R-1564-13
  TCGA-13-0797-01A-01R-1564-13 ... TCGA-13-1498-01A-01R-1565-13
  TCGA-31-1953-01A-01R-1568-13
colData names(63): sample patient ... releasable is_ffpe

In [12]:
proj_idx <- 3
q <- rna_seq_query(projects[proj_idx])
GDCdownload(q, method = "api", directory = TCGA_dest_dir, files.per.chunk = 10)
data <- GDCprepare(q, directory = TCGA_dest_dir)
data
saveHDF5SummarizedExperiment(data, dir = RSE_objects_dest_dir, prefix = paste0(projects[proj_idx], "_RNA_"))

--------------------------------------
o GDCquery: Searching in GDC database
--------------------------------------
Genome of reference: hg38
--------------------------------------------
oo Accessing GDC. This might take a while...
--------------------------------------------
ooo Project: TCGA-UCS
--------------------
oo Filtering results
--------------------
ooo By data.type
ooo By workflow.type
----------------
oo Checking data
----------------
ooo Check if there are duplicated cases
ooo Check if there results for the query
-------------------
o Preparing output
-------------------
GDCdownload will download 56 files. A total of 14.221611 MB


Downloading: 2.5 MB     

Downloading: 2.5 MB       

Downloading: 2.5 MB       

Downloading: 2.5 MB       

Downloading: 2.5 MB     

|====================================================|100%                      Completed after 5 s 


Starting to add information to samples
 => Add clinical information to samples
Add FFPE information. More information at: 
=> https://cancergenome.nih.gov/cancersselected/biospeccriteria 
=> http://gdac.broadinstitute.org/runs/sampleReports/latest/FPPP_FFPE_Cases.html
 => Adding subtype information to samples
ucs subtype information from:doi:10.1016/j.ccell.2017.02.010
Accessing www.ensembl.org to get gene information
Cache found
From the 60483 genes we couldn't map 3990


class: RangedSummarizedExperiment 
dim: 56493 56 
metadata(1): data_release
assays(1): HTSeq - Counts
rownames(56493): ENSG00000000003 ENSG00000000005 ... ENSG00000281912
  ENSG00000281920
rowData names(3): ensembl_gene_id external_gene_name
  original_ensembl_gene_id
colnames(56): TCGA-N5-A4RD-01A-11R-A28V-07 TCGA-NA-A5I1-01A-21R-A28V-07
  ... TCGA-N9-A4Q7-01A-11R-A28V-07 TCGA-NG-A4VW-01A-11R-A28V-07
colData names(128): sample patient ... subtype_histologic.subtype
  subtype_Estimated_Carc_percent

In [13]:
proj_idx <- 4
q <- rna_seq_query(projects[proj_idx])
GDCdownload(q, method = "api", directory = TCGA_dest_dir, files.per.chunk = 10)
data <- GDCprepare(q, directory = TCGA_dest_dir)
data
saveHDF5SummarizedExperiment(data, dir = RSE_objects_dest_dir, prefix = paste0(projects[proj_idx], "_RNA_"))

--------------------------------------
o GDCquery: Searching in GDC database
--------------------------------------
Genome of reference: hg38
--------------------------------------------
oo Accessing GDC. This might take a while...
--------------------------------------------
ooo Project: TCGA-UCEC
--------------------
oo Filtering results
--------------------
ooo By data.type
ooo By workflow.type
----------------
oo Checking data
----------------
ooo Check if there are duplicated cases
ooo Check if there results for the query
-------------------
o Preparing output
-------------------
GDCdownload will download 587 files. A total of 145.520507 MB


Downloading: 2.5 MB       

Downloading: 2.5 MB       

Downloading: 2.5 MB       

Downloading: 2.5 MB       

...

Downloading: 2.5 MB       

Downloading: 2.5 MB       

|====================================================|100%                      Completed after 50 s 


Starting to add information to samples
 => Add clinical information to samples
Add FFPE information. More information at: 
=> https://cancergenome.nih.gov/cancersselected/biospeccriteria 
=> http://gdac.broadinstitute.org/runs/sampleReports/latest/FPPP_FFPE_Cases.html
 => Adding subtype information to samples
ucec subtype information from:doi:10.1038/nature12113
Accessing www.ensembl.org to get gene information
Cache found
From the 60483 genes we couldn't map 3990


class: RangedSummarizedExperiment 
dim: 56493 587 
metadata(1): data_release
assays(1): HTSeq - Counts
rownames(56493): ENSG00000000003 ENSG00000000005 ... ENSG00000281912
  ENSG00000281920
rowData names(3): ensembl_gene_id external_gene_name
  original_ensembl_gene_id
colnames(587): TCGA-B5-A1MV-01A-31R-A14D-07
  TCGA-D1-A174-01A-11R-A12I-07 ... TCGA-FI-A2F4-01A-11R-A17B-07
  TCGA-AX-A3G7-01A-12R-A213-07
colData names(82): sample patient ... subtype_cna_cluster_k4
  subtype_mrna_expression_cluster

# Save data in count matrix + coldata format

Save data in a format ready for DESeq/clustering/etc.

In [22]:
data_ls <- load_RSE_objects(RSE_objects_dest_dir, projects, paste0(projects, "_RNA_"))

In [34]:
prep_and_save_count_data(
    rses = data_ls,
    label_field = "definition",
    dest_dir = dirs$data_dir,
    dest_subdir = "TCGA_RNA_matrix_count_data"
)